In [5]:
library(fastDummies)
library(cplm)
library(tweedie)
library(statmod)
library(HDtweedie)
library(TDboost)
library(mgcv)
library(MASS)
library(glmnet)

Loading required package: coda

Loading required package: Matrix

Loading required package: splines

Loading required package: lattice

Loaded TDboost 1.2


Loading required package: nlme


Attaching package: 'nlme'


The following objects are masked from 'package:cplm':

    fixef, ranef, VarCorr


This is mgcv 1.8-38. For overview type 'help("mgcv-package")'.

Loaded glmnet 4.1-2



In [78]:
x1<-rnorm(5000,4,2)
x2<-rnorm(5000,3,1)
x3<-rnorm(5000,1,0.5)

In [79]:
design_x <- as.data.frame(cbind(x1,x2,x3))
colnames(design_x) <- c('x1','x2','x3')

In [100]:
design_x_data <- model.matrix(~.+x1*x2-1,data=design_x)
beta_num<- c(0.2,0.25,0.3,0.2)
mu<-exp(design_x_data%*%beta_num)
y_sim<- rTweedie(mu,p=1.7,phi=15)

In [101]:
length(which(y_sim==0))

[1] 2305

In [102]:
sim_full<-cbind(design_x,y_sim)
colnames(sim_full)[4]='y'

In [103]:
sam <- sample(rep(1:2,len=5000))

In [104]:
sim_train<-sim_full[sam==1,]
sim_test<-sim_full[sam==2,]

In [105]:
length(which(sim_train$y==0))
length(which(sim_test$y==0))

[1] 1169

[1] 1136

In [106]:
sim_train<-as.data.frame(sim_train)
sim_test<-as.data.frame(sim_test)

In [107]:
fit1<-glm(y~.+x1*x2-1,data=sim_train,family=tweedie(link.power=0,var.power=1.7))
summary(fit1)
mean(residuals(fit1)^2)


Call:
glm(formula = y ~ . + x1 * x2 - 1, family = tweedie(link.power = 0, 
    var.power = 1.7), data = sim_train)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-11.1448   -4.3211   -3.2978   -0.0382   12.7976  

Coefficients:
      Estimate Std. Error t value Pr(>|t|)    
x1    0.159169   0.027577   5.772 8.82e-09 ***
x2    0.188299   0.035149   5.357 9.23e-08 ***
x3    0.368154   0.074804   4.922 9.14e-07 ***
x1:x2 0.216484   0.008197  26.411  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Tweedie family taken to be 15.74616)

    Null deviance: 28641453  on 2500  degrees of freedom
Residual deviance:    38178  on 2496  degrees of freedom
AIC: NA

Number of Fisher Scoring iterations: 6


[1] 15.27105

In [108]:
shim <- model.matrix(~.+x1*x2+x1*x3+x2*x3,data=sim_train)

In [109]:
#Initial: For example, we can use the least square estimates or the simple regression estimates by regressing the response y on each of the terms.
# Main effects
fit_b1<-glm(y~x1-1,data=sim_train,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
b1_0 <- fit_b1$coefficients

fit_b2<-glm(y~x2-1,data=sim_train,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
b2_0 <- fit_b2$coefficients

fit_b3<-glm(y~x3-1,data=sim_train,family=tweedie(link.power=0,var.power=1.7))
b3_0 <- fit_b3$coefficients

In [110]:
#Interaction: when update r12, use y~b1b2x1x2 with no intercept.
effect_12<-b1_0*b2_0*sim_train$x1*sim_train$x2
fit_r12<-glm(sim_train$y~effect_12-1,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
r12_0<-fit_r12$coefficients

effect_13<-b1_0*b3_0*sim_train$x1*sim_train$x3
fit_r13<-glm(sim_train$y~effect_13-1,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
r13_0<-fit_r13$coefficients

effect_23<-b1_0*b2_0*sim_train$x2*sim_train$x3
fit_r23<-glm(sim_train$y~effect_23-1,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
r23_0<-fit_r23$coefficients

In [112]:
b1_0
b2_0
b3_0
r12_0
r13_0
r23_0

In [121]:
X1<-sim_train$x1
X2<-sim_train$x2
X3<-sim_train$x3

In [157]:
#Update r: y~r12 (b1b2x1x2+r13b1b3x1x3+r23b2b3x2x3)+ offset [b1x1+b2x2+b3x3]
#offset:
offset_r_1<-b1_0*X1+b2_0*X2+b3_0*X3

In [158]:
effect_1_matrix<- as.matrix(cbind(effect_12,effect_13,effect_23))

In [159]:
#Use lambda from r12 as candidate
r12_1_fit<-cv.glmnet(x=effect_1_matrix,y=sim_train$y,family=tweedie(link.power=0,var.power=1.7),offset=offset_r_1,intercept=FALSE,type.measure = 'deviance')

In [160]:
lambda_r<-r12_1_fit$lambda

In [171]:
r_update<-coef(r12_1_fit,s='lambda.min')[,1]
r_update_1<-r_update[2:length(r_update)]
r_update_1

effect_12   effect_13   effect_23 
-0.04972602 -0.02468582 -0.61372864

In [ ]:
max( abs(t(Y - mean(Y)*(1-mean(Y))) %*% X ) )/ ( alpha * n)

In [173]:
#Update b1. To note design matrix is (0, x1)
offset_b1<-b2_0*X2+b3_0*X3+b2_0*b3_0*X2*X3
b1_x1<-X1+r_update_1[1]*b2_0*X1*X2+r_update_1[2]*b3_0*X1*X3
b1_matrix <- (as.matrix(cbind(0,b1_x1)))
b1_1_fit<-cv.glmnet(x=b1_matrix,y=sim_train$y,family=tweedie(link.power=0,var.power=1.7)
                    ,offset=offset_b1,intercept=FALSE,type.measure = 'deviance')
b1_update<-coef(b1_1_fit,s='lambda.min')[,1]
b1_update_1<-b1_update[3]

In [213]:
length(offset_b2)

[1] 2500

In [210]:
#Update b2
offset_b2<-b1_update_1*X1+b3_0*X3+b1_update_1*b3_0*X1*X3
b2_x1<-X2+r_update_1[1]*b1_update_1*X1*X2+r_update_1[3]*b3_0*X2*X3
b2_matrix <- as.matrix(cbind(0,b2_x1))
b2_1_fit<-cv.glmnet(x=b2_matrix,y=sim_train$y,family=tweedie(link.power=0,var.power=1.7)
                    ,offset=offset_b2,intercept=FALSE,type.measure = 'deviance')
# b2_update<-coef(b2_1_fit,s='lambda.min')[,1]
# b2_update_1<-b2_update[3]

ERROR: Error in elnet.fit(x, z, w, lambda, alpha, intercept, thresh = thresh, : NA/NaN/Inf in foreign function call (arg 8)


In [182]:
#Update b3
offset_b3<-b1_0*X1+b2_0*X2+b1_0*b2_0*X1*X2
b3_x1<-X3+r_update_1[2]*b1_0*X1*X3+r_update_1[3]*b2_0*X2*X3
b3_matrix <- (as.matrix(cbind(0,b3_x1)))
b3_1_fit<-cv.glmnet(x=b3_matrix,y=sim_train$y,family=tweedie(link.power=0,var.power=1.7)
                    ,offset=offset_b3,intercept=FALSE,type.measure = 'deviance')
b3_update<-coef(b3_1_fit,s='lambda.min')[,1]
b3_update_1<-b3_update[3]

In [183]:
b_update_1<-c(b1_update_1,b2_update_1,b3_update_1)

In [185]:
b_update_1

b1_x1      b2_x1      b3_x1 
-15.021420   1.650195   2.466912

In [198]:
lambda_b_1<-c(b1_1_fit$lambda.min,b2_1_fit$lambda.min,b3_1_fit$lambda.min)

In [188]:
X<-as.matrix(cbind(X1,X2,X3))

In [203]:
fitted_1 <- X%*%b_update_1 
Q_1 <- -sum(tweedie.dev(sim_train$y,fitted_1,1.7),na.rm=T)+sum(abs(r_update_1))*r12_1_fit$lambda.min+t(abs(b_update_1))%*%lambda_b_1